In [1]:
import torch
import pickle
import numpy as np
import sys
sys.path.insert(1, 'src/agent/')

/home/ark/miniconda3/envs/pvm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEMO_FILE = "expert_demos/kitchen/kitchen_knob1_on-v3/expert_demos.pkl"
device = "cuda:0"

In [3]:
imgs, states, actions, rewards = pickle.load(open(DEMO_FILE, 'rb'))

data-format:
- imgs: (demo_id, timestep, frame_stacks * 3, 244, 244)
- state: (demo_id, timestep, state)
- actions: (demo_id, timestep, action)
- rewards: (demo_id, timestep)

In [4]:
from resnet import get_unsupervised_resnet
encoder, repr_dim = get_unsupervised_resnet(".", "r3m-resnet50", [], False)
encoder.to(device)
encoder.freeze()
encoder.eval()

All keys matched successfully!
Loaded encoder from: ./pretrained/original_r3m.pt
Trainable parameters in the encoder: []


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
img_reprs = []
for demo in imgs:
    with torch.cuda.amp.autocast(enabled=True):
        repr = encoder(torch.as_tensor(demo, device=device))
    img_reprs.append(repr.float().cpu().numpy())
img_reprs = np.stack(img_reprs, axis=0)

In [8]:
print(img_reprs.shape, actions.shape)
n_train = int(0.9 * img_reprs.shape[0])
action_dim = actions.shape[-1]

# Datasets
X_train = img_reprs[:n_train].reshape((-1, repr_dim))
X_val = img_reprs[n_train:].reshape((-1, repr_dim))

y_train = actions[:n_train].reshape((-1, action_dim))
y_val = actions[n_train:].reshape((-1, action_dim))

(50, 50, 2048) (50, 50, 9)


In [21]:
# installed `conda install scikit-learn`
from sklearn.decomposition import PCA

pca = PCA(n_components=16)
pca.fit(X_train)

PCA(n_components=16)

In [22]:
print("Explained Variance: ", pca.explained_variance_ratio_)
print("Singular values: ", pca.singular_values_)

Explained Variance:  [0.575763   0.13352613 0.08800177 0.06641862 0.02733543 0.01899836
 0.01439553 0.01273793 0.01052288 0.00806019 0.00575794 0.0041022
 0.00368743 0.00288451 0.00253714 0.00218334]
Singular values:  [8.073135   3.8877964  3.1562111  2.7419863  1.7590709  1.4664875
 1.2765394  1.2007977  1.0914094  0.9551973  0.80733514 0.68144196
 0.6460739  0.57142127 0.5359109  0.49714234]


In [23]:
import matplotlib.pyplot as plt

In [24]:
plt.imshow(pca.components_ / pca.components_.max())
plt.axis('off')
plt.show()